In [ ]:
# test
import pickle
import pandas as pd
import numpy as np

# load data (enter the path to the file of data to be tested here)
df_test=pd.read_csv("testing_data_sentiment.csv")

# cleaning 
def  clean_text(df, text_field):
    import re
    file=open("vietnamese-stopwords.txt")
    stp=file.read().split("\n")
    pat = r'\b(?:{})\b'.format('|'.join(stp))
    for i in text_field:
        df[i] = df[i].apply(lambda elem: re.sub(r"[\|]", "", elem))  
        df[i] = df[i].str.replace(pat, '')
    return df

def pos(df, text_field, col):
    from pyvi import ViTokenizer, ViPosTagger
    m=[]
    for i in range(len(df)):
        q_pos=ViPosTagger.postagging(ViTokenizer.tokenize(df[text_field].iloc[i]))
        dict_q_pos=dict(zip(q_pos[0], q_pos[1]))
        lst=[x for x, y in dict_q_pos.items() if y in ["N", "Ny", "Np", "Nu", "Nc", "A", "V"]]
        l=[["".join(x) for x in lst]]
        m.append([" ".join(x) for x in l])
    return pd.DataFrame(m, columns=[col])

clean_text(df_test, ["question", "answer"])
df_test["q_pos"]=pos(df_test,"question", "q_pos")
df_test["a_pos"]=pos(df_test,"answer", "a_pos")


# test
xgb_model = pickle.load(open('xgb_model.pckl', 'rb'))
y_pred=xgb_model.predict(df_test['q_pos']+df_test['a_pos'])
y_pred=np.where(y_pred==0, "N", "Y")
df_test=pd.DataFrame({"num":df_test.index+1, "label":y_pred})
df_test.to_csv("Sam_submission.csv", index=False)